In [2]:
import pandas as pd
from backtesting import Strategy
from backtesting.lib import crossover, TrailingStrategy
from backtesting import Backtest
from backtesting.test import SMA

import numpy as np


data = pd.read_csv("../data/BTC-USD.csv", parse_dates=[0], date_parser=pd.to_datetime, index_col = [0], header = 0)


In [3]:
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 5
    n2 = 8
    sltr=500e-4

    def init(self):
        # Precompute the two moving averages
        super().init()
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        
    
    def next(self):

        super().next()

        sltr = self.sltr
        for trade in self.trades: 
            if trade.is_long: 
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - sltr)
                print("entry_time: " + str(trade.entry_time) + " ,sl: " + str(trade.sl))
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + sltr) 

        if (self.data.Open[-1] > self.sma1[-2] and 
        self.data.Open[-1] > self.sma2[-2] and
        self.data.Open[-1] > self.data.Low[-2] and 
        self.data.Low[-2] > self.data.Low[-3] and
        self.data.Low[-3] > self.data.Low[-4]):
            print ("self.data.Open[-1]: " + str(self.data.Open[-1]) + " self.sma1[-2]: " + str(self.sma1[-2]) + "")
            self.buy(limit=self.data.Open[-1], sl=self.data.Low[-2])


In [4]:
bt = Backtest(data, SmaCross, cash=1000000, commission=.002)
stats = bt.run()
stats
# stats['_equity_curve']


self.data.Open[-1]: 12889.200195 self.sma1[-2]: 11732.1201172
entry_time: 2018-01-22 00:00:00 ,sl: 11656.200195
self.data.Open[-1]: 11755.5 self.sma1[-2]: 11403.440234599999
entry_time: 2018-01-30 00:00:00 ,sl: 11475.299805
self.data.Open[-1]: 8271.839844 self.sma1[-2]: 7774.633887000001
self.data.Open[-1]: 8720.080078 self.sma1[-2]: 7866.628027599999
entry_time: 2018-02-11 00:00:00 ,sl: 8129.920215
self.data.Open[-1]: 8616.129883 self.sma1[-2]: 8199.9541018
entry_time: 2018-02-11 00:00:00 ,sl: 8926.520313
self.data.Open[-1]: 9488.320313 self.sma1[-2]: 8754.2760744
self.data.Open[-1]: 10135.700195 self.sma1[-2]: 9063.1760744
self.data.Open[-1]: 10207.5 self.sma1[-2]: 9483.9621096
self.data.Open[-1]: 11123.400391 self.sma1[-2]: 9921.188085999998
entry_time: 2018-02-19 00:00:00 ,sl: 11225.249805000001
self.data.Open[-1]: 10552.599609 self.sma1[-2]: 10311.886133
entry_time: 2018-02-19 00:00:00 ,sl: 11403.650195
self.data.Open[-1]: 11231.799805 self.sma1[-2]: 10658.0201174
entry_time: 2018

/home/codespace/.python/current/lib/python3.10/site-packages/backtesting/backtesting.py:912: UserWarning: (2018-01-22 00:00:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
  warnings.warn(
/home/codespace/.python/current/lib/python3.10/site-packages/backtesting/backtesting.py:912: UserWarning: (2018-03-06 00:00:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
  w

Start                     2018-01-01 00:00:00
End                       2022-11-17 00:00:00
Duration                   1781 days 00:00:00
Exposure Time [%]                   34.567901
Equity Final [$]               1756708.147604
Equity Peak [$]                2646662.378044
Return [%]                          75.670815
Buy & Hold Return [%]               21.742974
Return (Ann.) [%]                   12.233073
Volatility (Ann.) [%]               40.803771
Sharpe Ratio                         0.299802
Sortino Ratio                        0.597971
Calmar Ratio                         0.302332
Max. Drawdown [%]                  -40.462408
Avg. Drawdown [%]                   -6.100198
Max. Drawdown Duration     1016 days 00:00:00
Avg. Drawdown Duration       71 days 00:00:00
# Trades                                  390
Win Rate [%]                        51.025641
Best Trade [%]                       33.57372
Worst Trade [%]                    -10.779139
Avg. Trade [%]                    

In [5]:
bt.plot()

/home/codespace/.python/current/lib/python3.10/site-packages/bokeh/models/formatters.py:399: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first prodvided: '%d %b' 
  warnings.warn(f"DatetimeFormatter scales now only accept a single format. Using the first prodvided: {fmt[0]!r} ")
/home/codespace/.python/current/lib/python3.10/site-packages/bokeh/models/formatters.py:399: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first prodvided: '%m/%Y' 
  warnings.warn(f"DatetimeFormatter scales now only accept a single format. Using the first prodvided: {fmt[0]!r} ")


GridPlot(id='p1730', ...)